### **Tensorflow9강 Constructing CNN final**

In [1]:
import tensorflow as tf
import os
from PIL import Image  # image 출력을 위한 library

In [8]:
image_dir = os.getcwd() + "/test_image/dataset/"
image_list = os.listdir(image_dir)
image_list.sort()

for i in range(len(image_list)):
    image_list[i] = image_dir + image_list[i]

label_dir = os.getcwd() + "/test_image/label.csv"
label_list = [label_dir]

image_queue = tf.train.string_input_producer(image_list)
label_queue = tf.train.string_input_producer(label_list)

image_reader = tf.WholeFileReader()
label_reader = tf.TextLineReader()

image_key, image_value = image_reader.read(image_queue)
label_key, label_value = label_reader.read(label_queue)

image_decoded = tf.image.decode_png(image_queue)
label_decoded = tf.decode_csv(label_value, record_defaults=[[0]])

batch_image, batch_label = tf.train.shuffle_batch(tensor=[image_decoded, label_decoded], batch_size=32, num_threads=4, capacity=50000, min_after_dequeue=100, enqueue_many=True)

x = tf.cast(image, tf.float32)
y_ = tf.cast(label, tf.float32)
y_ = tf.reshape(y_, [-1, 1])

TypeError: Expected string passed to parameter 'contents' of op 'DecodePng', got <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x7f7b12f6a5c0> of type 'FIFOQueue' instead.

In [ ]:
IMAGE_WIDTH =50
IMAGE_HEIGHT = 50
#HIDDEN1_SIZE = 32
#HIDDEN2_SIZE = 64
#CLASSES = 10

In [ ]:
hidden1_w = tf.Variable(tf.truncated_normal([5, 5, 1, 32]))  # width, height, depth, batch
hidden1_b = tf.Variable(tf.zeros([32]))

hidden2_w = tf.Variable(tf.truncated_normal([5, 5, 32, 64]))
hidden2_b = tf.Variable(tf.zeros([64]))


fc_w = tf.Variable(tf.truncated_normal([IMAGE_WIDTH * IMAGE_HEIGHT * 64, 10]))  # CLASS는 임의의 숫자 
fc_b = tf.Variable(tf.zeros([10]))

out_w = tf.Variable(tf.truncated_normal([10, 1]))
out_b = tf.Variable(tf.zeros([1]))

In [ ]:
# convolution
x_image = tf.reshape(x, [-1, IMAGE_WIDTH, IMAGE_HEIGHT, 1])  # reshape를 해주어야 dimension오류가 없다.

h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, hidden1_w, strides=[1, 1, 1, 1], padding="SAME") + hidden1_b)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, hidden2_w, strides=[1, 1, 1, 1], padding="SAME") + hidden2_b)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

In [ ]:
# fully connected layer
h_flat = tf.reshape(h_pool2, [-1, IMAGE_WIDTH * IMAGE_HEIGHT * 64])
h_fc1 = tf.nn.softmax(tf.matmul(h_flat, fc_w) + fc_b)

drop_fc = tf.nn.dropout(h_fc1, 0.7)
pred = tf.matmul(drop_fc, out_w) + out_b

In [ ]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y_))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [ ]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))  # 1: reduce할 때, 어느 방향으로 할꺼
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
 with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    sess.run(tf.global_variables_initializer())
    
    for i in range(10):
        sess.run(train)
        _loss, _accuracy = sess.run([loss, accuracy])
        print("=========================")
        print("loss: ", _loss)
        print("accuracy: ", _accuracy)
    
    coord.request_stop()
    coord.join(threads)